In [1]:
import os
from pathlib import Path
import pandas as pd

In [2]:
cwd =os.getcwd()
p_cwd = Path(cwd).parent.absolute()


In [3]:
stations_loc = pd.read_csv(os.path.join(p_cwd, "output","printouts","de_phen_stations.csv"), index_col=None)

In [4]:
stations_loc

,s_id,lon,lat,name
0,1,9.43333,54.7667,"Flensburg, kreisfreie Stadt Flensburg"
1,2,10.15000,54.4000,Kiel-Pries
2,3,10.15000,54.3333,Kiel-Düsternbrook
3,4,10.05000,54.3167,Kiel-Mettenhof
4,5,10.68330,53.8833,Lübeck-Schlutup
...,...,...,...,...
6768,21528,14.06670,52.5500,Buckow/Märkische Schweiz
6769,21529,12.10000,54.1000,Rostock-Gehlsdorf
6770,21530,12.23330,51.8000,Dessau-Süd
6771,21531,10.40000,50.7500,Heßles


In [5]:
stations_info = pd.read_csv(os.path.join(p_cwd, "output","printouts","stations_info.csv"), index_col=None)

In [6]:
stations_info

,s_id,total_obs,time_frame
0,1,2515,1954-1999
1,2,2176,1959-1998
2,3,614,1983-2009
3,4,1686,1990-2015
4,5,308,1978-1995
...,...,...,...
6768,21528,21,2019-2019
6769,21529,58,2019-2019
6770,21530,27,2019-2019
6771,21531,39,2019-2019


In [7]:
stations = pd.merge(stations_loc, stations_info, on='s_id' )

In [8]:
stations.head()

,s_id,lon,lat,name,total_obs,time_frame
0,1,9.43333,54.7667,"Flensburg, kreisfreie Stadt Flensburg",2515,1954-1999
1,2,10.15000,54.4000,Kiel-Pries,2176,1959-1998
2,3,10.15000,54.3333,Kiel-Düsternbrook,614,1983-2009
3,4,10.05000,54.3167,Kiel-Mettenhof,1686,1990-2015
4,5,10.68330,53.8833,Lübeck-Schlutup,308,1978-1995


In [9]:
stations.dtypes

s_id            int64
lon           float64
lat           float64
name           object
total_obs       int64
time_frame     object
dtype: object

In [10]:
import folium
from folium.plugins import MarkerCluster

In [11]:
SF_COORDINATES = (52.520008, 13.404954) #Berlin coordinates

In [12]:
# create empty map zoomed in Berlin
stations_map = folium.Map(location=SF_COORDINATES, zoom_start=10, tiles="Stamen Terrain")
marker_cluster = MarkerCluster().add_to(stations_map)

for station in stations.iterrows():
    tooltip = "Station info"
    
    popup_string = '{}, {} observations in {}'.format(station[1]['name'],
                                                                 station[1]['total_obs'],
                                                                 station[1]['time_frame'] )
    
    folium.Marker([station[1]['lat'],station[1]['lon']],
                  popup=popup_string, icon=folium.Icon(color="green", icon="suit-diamond-fill"), 
                  tooltip=tooltip).add_to(marker_cluster)

In [13]:
stations_map.save(outfile= os.path.join(p_cwd, "output","images","phenology_stations_map.html"))

In [ ]:
import os
from pathlib import Path
import geopandas as gpd
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('seaborn')

In [ ]:
%matplotlib inline

In [ ]:
cwd =os.getcwd()
p_cwd = Path(cwd).parent.absolute()
df_stations = pd.read_csv(os.path.join(p_cwd,"data","de_phen_stations.csv"))

In [ ]:
df_stations.head()

In [ ]:
gdf = gpd.GeoDataFrame(
     df_stations, geometry=gpd.points_from_xy(df_stations.lon, df_stations.lat))

In [ ]:
gdf.crs = "WGS84"

In [ ]:
gdf

In [ ]:
gdf_trans = gdf.to_crs(epsg=3857)

In [ ]:
gdf_trans.plot()

In [ ]:
gdf_trans

In [ ]:
gdf_trans.describe()

In [ ]:
ctx.providers.keys()

In [ ]:
fig, ax = plt.subplots(figsize=(30, 30))
extent = (600000, 1700000, 5900000, 7400000)
gdf_trans.plot(ax=ax, color='red', markersize=20.0)
ax.axis(extent)
ctx.add_basemap(ax, source=ctx.providers.OpenTopoMap)

plt.savefig(os.path.join(p_cwd, "output", "images", "map_phen_stat_de.jpg"), dpi=300, facecolor='w', edgecolor='w',
        orientation='landscape', papertype=None, format='jpg',
        transparent=True, bbox_inches=None, pad_inches=0.1,
        frameon=None, metadata=None)
